# Install

In [4]:
%pip install langchain===0.0.200
%pip install openai
%pip install python-dotenv
%pip install google-search-results


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32002 sha256=bc3a79c1abcc5d8744e4702fdb80de121ef3da3d444832dd70fb08feebd9f068
  Stored in directory: /home/codespace/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, ru

# Setup
This is using 2 deployments(chat and completion)


In [6]:
import os
import openai
from dotenv import load_dotenv

# Load environment variables from a .env file using load_dotenv():
load_dotenv()

azure_openai_api_key: str = os.environ.get('AZURE_OPENAI_API_KEY')
deployment_id='VERX-CORP-GPT40'
completion_deployment_id='VERX-CORP-DAVINCI'

openai.api_type = "azure"
openai.api_base = "https://verx-corp-ai.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = azure_openai_api_key
openai.openai_api_key = azure_openai_api_key
openai.model_name = 'gpt-4-32k'
openai.Deployment = 'VERX-CORP-GPT40'

# The Agent


The Agent is using the google search api to search for the relevant "live" information and then using the calculator tool to make calculations if need be.

The example here is asking it to find out what the current sales tas in atlanta is and calcualte the total cost including sales tax

In [10]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0, openai_api_key=azure_openai_api_key, deployment_id=deployment_id)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0, openai_api_key=azure_openai_api_key, deployment_id=completion_deployment_id)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Run the agent
agent.run("I want to buy candybar in Atlanta, Georiga for $3.00.  What would the total cost be including sales tax?")

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.
WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.




> Entering new  chain...
Thought: I need to find the sales tax rate in Atlanta, Georgia.
Action:
```
{
  "action": "Search",
  "action_input": "sales tax rate in Atlanta Georgia"
}
```
Observation: 8.9%
Thought:I now know the sales tax rate in Atlanta, Georgia is 8.9%. I need to calculate the total cost of the candy bar including sales tax.
Action:
```
{
  "action": "Calculator",
  "action_input": "3 * (1 + 0.089)"
}
```

Observation: Answer: 3.267
Thought:I now know the final answer.
Final Answer: The total cost of the candy bar in Atlanta, Georgia, including sales tax, would be $3.27.

> Finished chain.


'The total cost of the candy bar in Atlanta, Georgia, including sales tax, would be $3.27.'